<a href="https://colab.research.google.com/github/beaumontslim/misc/blob/main/txt_classfification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import nltk

nltk.download('all')

In [25]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/Restaurant_Reviews.tsv.txt', delimiter= '\t', quoting = 3)

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

corpus = []

import re

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
 1   Liked   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [26]:
for i in range(0, 1000):

  customer_review = re.sub('[^a-zA-Z]', ' ',dataset['Review'][i])
  customer_review = customer_review.lower()
  customer_review = customer_review.split()
  clean_review = [ps.stem(word) for word in customer_review if not word in set(stopwords.words('english'))]
  clean_review = ' '.join(clean_review)
  corpus.append(clean_review)

In [28]:
#corpus[0]

corpus[9]

#corpus[12]

'great touch'

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 1500, min_df = 3, max_df = 0.6)

X = vectorizer.fit_transform(corpus).toarray()

X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
y = dataset.iloc[:, 1].values

y[0]

1

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

import torch
import torch.nn as nn
from torch.nn import functional as F


In [37]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

Xtrain_.shape, ytrain_.shape

(torch.Size([800, 467]), torch.Size([800]))

In [38]:
Xtest_.shape, ytest_.shape

(torch.Size([200, 467]), torch.Size([200]))

In [39]:
#input suze reflects shape of Xtest shape, feeds to two output neurons (for 1/0)

input_size = 467
output_size = 2
hidden_size = 500

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)

  def forward (self, X):
    X = torch.relu((self.fc1(X)))
    X = torch.relu((self.fc2(X)))
    X = self.fc3(X)

    return F.log_softmax(X,dim=1)

model = Net()

import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr = .01)
loss_fn = nn.NLLLoss()



epochs = 100

In [40]:
for epoch in range (epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred, ytrain_)
  loss.backward()
  optimizer.step()
  print('Epock! Text_Class_PyTorch', epoch, 'loss',loss.item())

Epock! Text_Class_PyTorch 0 loss 0.6934424042701721
Epock! Text_Class_PyTorch 1 loss 0.6767289042472839
Epock! Text_Class_PyTorch 2 loss 0.5655756592750549
Epock! Text_Class_PyTorch 3 loss 0.3697829842567444
Epock! Text_Class_PyTorch 4 loss 0.24900339543819427
Epock! Text_Class_PyTorch 5 loss 0.171419158577919
Epock! Text_Class_PyTorch 6 loss 0.11265963315963745
Epock! Text_Class_PyTorch 7 loss 0.09049338102340698
Epock! Text_Class_PyTorch 8 loss 0.07926288992166519
Epock! Text_Class_PyTorch 9 loss 0.06423422694206238
Epock! Text_Class_PyTorch 10 loss 0.05173061415553093
Epock! Text_Class_PyTorch 11 loss 0.04910440370440483
Epock! Text_Class_PyTorch 12 loss 0.04611667990684509
Epock! Text_Class_PyTorch 13 loss 0.04041260853409767
Epock! Text_Class_PyTorch 14 loss 0.03530469909310341
Epock! Text_Class_PyTorch 15 loss 0.034949686378240585
Epock! Text_Class_PyTorch 16 loss 0.036952532827854156
Epock! Text_Class_PyTorch 17 loss 0.0324549563229084
Epock! Text_Class_PyTorch 18 loss 0.0317455

In [43]:
sample = ['good batting by America']
sample = vectorizer.transform(sample).toarray()
#sample
torch.from_numpy(sample).float()
sentiment = model(torch.from_numpy(sample).float())
sentiment

tensor([[-1.6025, -0.2249]], grad_fn=<LogSoftmaxBackward>)

In [44]:
sample = ['bad performance by the jayhawks, no sweet sixteen! :/']
sample = vectorizer.transform(sample).toarray()
#sample
torch.from_numpy(sample).float()
sentiment = model(torch.from_numpy(sample).float())
sentiment

tensor([[-0.5914, -0.8064]], grad_fn=<LogSoftmaxBackward>)

In [45]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[-0.0893, -0.0185, -0.1271,  ...,  0.1424,  0.1543, -0.0721],
                      [-0.0389, -0.1162,  0.0771,  ..., -0.0181,  0.1433, -0.1334],
                      [ 0.0865,  0.0620, -0.0222,  ...,  0.1309,  0.0421,  0.1253],
                      ...,
                      [ 0.0093, -0.0429,  0.0986,  ...,  0.0260,  0.1005, -0.0716],
                      [-0.0145, -0.0315, -0.0855,  ...,  0.0419, -0.0048, -0.0910],
                      [-0.1347, -0.0834,  0.1154,  ...,  0.0926,  0.0145, -0.1321]])),
             ('fc1.bias',
              tensor([-0.0448,  0.0414, -0.0205, -0.0422, -0.0411,  0.0541, -0.0143,  0.0308,
                      -0.0878, -0.0755, -0.0838, -0.0141, -0.0367, -0.0905,  0.0423, -0.0171,
                       0.0053, -0.1174,  0.0052, -0.0041, -0.0265,  0.0075, -0.0253,  0.0251,
                       0.0343,  0.0048, -0.1012, -0.0409,  0.0082, -0.1144, -0.0577, -0.1123,
                      -0.0307, -0.01

In [46]:
torch.save(model.state_dict(),'text_class_pytorch')

In [47]:
!ls

sample_data  text_class_pytorch


In [48]:
from google.colab import files

In [49]:
files.download('text_class_pytorch')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
import pickle
with open ('tfidfmodel.pickle', 'wb') as file:
  pickle.dump(vectorizer,file)